In [1]:
!pip install --upgrade --quiet  langchain langchain-community langchain-groq neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.8/304.8 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.9/241.9 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.5/599.5 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.

In [2]:
# Wait 60 seconds before connecting using these details, or login to https://console.neo4j.io to validate the Aura Instance is available
NEO4J_URI="neo4j+s://6ed9b7da.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="14V4z2NxsI8laZL3tvaAvyJzIdgyZCwgxKHbRWObatM"


In [3]:
import os
os.environ["NEO4J_URI"]=NEO4J_URI
os.environ["NEO4J_USERNAME"]=NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"]=NEO4J_PASSWORD

In [4]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
)

In [5]:
graph

In [6]:
groq_api_key="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"

In [7]:
from langchain_groq import ChatGroq

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7dff89382cb0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7dff893816f0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [8]:
!pip install --upgrade --quiet langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 3.5 MB/s eta 0:00:00


In [9]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer=LLMGraphTransformer(llm=llm)

In [11]:
graph

In [13]:
graph.refresh_schema()
print(graph.schema)

Node properties:
CVE {id: STRING, cvss_v2: FLOAT, severity: STRING, cwe_id: STRING, software_component: STRING, impact: STRING, network_requirements: STRING, privileges_required: STRING, affected_protocols: STRING, authentication_required: STRING, user_interaction_required: STRING, vendor: STRING, operating_system: STRING, version: STRING, affected_hardware: STRING, cvss_v3: FLOAT}
Protocol {name: STRING}
Relationship properties:

The relationships:
(:CVE)-[:AFFECTS_PROTOCOL]->(:Protocol)


# Iske Niche wali dono Cell dekh

In [52]:
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(llm=llm,graph=graph,verbose=True, allow_dangerous_requests=True, return_intermediate_steps=True) # added return_intermediate_steps=True
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x7dffbb1a5de0>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7dff89382cb0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7dff893816f0>, model_name='Gemma2-9b-I

In [60]:
response=chain.invoke({"query":"Show me vulnerabilities that affect the TCP/IP protocol."})
response
print(f"Intermediate steps: {response['intermediate_steps']}")  # giving full context.



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:CVE)-[:AFFECTS_PROTOCOL]->(p:Protocol) WHERE p.name = 'TCP/IP' RETURN c

Full Context:
[{'c': {'severity': 'MEDIUM', 'cwe_id': 'CWE-20', 'cvss_v2': 5.0, 'impact': 'denial of service (crash or hang)', 'operating_system': 'BSD-derived', 'id': 'CVE-1999-0001', 'software_component': 'ip_input.c', 'network_requirements': 'remote', 'affected_protocols': 'TCP/IP'}}, {'c': {'severity': 'HIGH', 'cwe_id': 'NVD-CWE-Other', 'vendor': 'Foundry Networks', 'cvss_v2': 7.5, 'impact': 'spoof or hijack sessions', 'affected_hardware': 'ServerIron switches', 'id': 'CVE-2000-0178', 'software_component': 'ServerIron switches', 'network_requirements': 'remote', 'affected_protocols': 'TCP/IP'}}, {'c': {'severity': 'LOW', 'cwe_id': 'NVD-CWE-Other', 'vendor': 'Microsoft', 'cvss_v2': 2.1, 'impact': 'denial of service', 'operating_system': 'Windows', 'id': 'CVE-2000-0232', 'software_component': 'TCP/IP Printing Services, aka Print Services for

In [29]:
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(llm=llm,graph=graph,verbose=True, allow_dangerous_requests=True)

response=chain.invoke({"query":"What are the details for CVE-1999-0013?"})




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:CVE {id: 'CVE-1999-0013'}) RETURN c

Full Context:
[{'c': {'severity': 'HIGH', 'cwe_id': 'NVD-CWE-Other', 'cvss_v2': 7.5, 'impact': 'other local users can access remote accounts', 'id': 'CVE-1999-0013', 'software_component': 'ssh-agent program', 'network_requirements': 'local', 'affected_protocols': 'SSH', 'authentication_required': 'Yes'}}]

> Finished chain.


In [20]:
def get_full_context(response):
    # Extract 'Full Context' key from the response if it exists
    return response.get('Full Context', [])


In [21]:
full_context_json = get_full_context(response)
print(full_context_json)


[]


In [23]:
# Print the keys of the response dictionary to check its structure
print(response.keys())

# Print the entire response dictionary for further inspection
print(response)


dict_keys(['query', 'result'])
{'query': 'What are the details for CVE-1999-0013?', 'result': 'The severity is HIGH, the CWE ID is NVD-CWE-Other, the CVSS v2 score is 7.5, the impact is that other local users can access remote accounts, and it affects the ssh-agent program. It requires authentication and uses SSH, with local network requirements.  \n'}


In [30]:
import io
import contextlib

# Capture the verbose output
output = io.StringIO()
with contextlib.redirect_stdout(output):
    response = chain.invoke({"query": "What are the details for CVE-1999-0013?"})

# Get the captured output
verbose_output = output.getvalue()




In [31]:
# Extract the Full Context from the verbose output (this may vary based on the output format)
import re
import ast

# Regular expression to find "Full Context" section
full_context_match = re.search(r"Full Context:\n(\[.*\])", verbose_output, re.DOTALL)

print(full_context_match)

None


In [51]:
import json

response = chain.invoke({"query": "What are the details for CVE-1999-0013?"})
print(response)

# Search for the "Full Context:" marker in the result string
full_context_start = response['result'].find('Full Context:')

if full_context_start != -1:
    # Extract the JSON data following the "Full Context:" marker
    full_context_json_str = response['result'][full_context_start + len('Full Context:'):].strip()
    full_context_json = json.loads(full_context_json_str)
    print(full_context_json)
else:
    print("Full context information not found in the response.")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:CVE {id: 'CVE-1999-0013'}) RETURN c

Full Context:
[{'c': {'severity': 'HIGH', 'cwe_id': 'NVD-CWE-Other', 'cvss_v2': 7.5, 'impact': 'other local users can access remote accounts', 'id': 'CVE-1999-0013', 'software_component': 'ssh-agent program', 'network_requirements': 'local', 'affected_protocols': 'SSH', 'authentication_required': 'Yes'}}]

> Finished chain.
{'query': 'What are the details for CVE-1999-0013?', 'result': 'The severity of CVE-1999-0013 is HIGH. Its CWE ID is NVD-CWE-Other, it has a CVSS v2 score of 7.5, and its impact is that other local users can access remote accounts. It affects the ssh-agent program and requires authentication.  It is related to the SSH protocol and requires local network access. \n'}
Full context information not found in the response.


In [43]:
json_only_chain = chain.pick("Full Context:")

In [47]:
print(json_only_chain.invoke)

<bound method RunnableSequence.invoke of GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x7dffbb1a5de0>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7dff89382cb0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at